# Sleaper API Relative Endpoints

In [0]:
# fetch parameters
par_league_id = dbutils.widgets.get("LEAGUE_ID")
par_matchup_week = int( dbutils.widgets.get("MATCHUP_WEEK") )
par_year = dbutils.widgets.get("YEAR")

In [0]:
%run /Workspace/Users/djamsheedy99@gmail.com/SleeperWarehouse/src/ingest/_utils_sleeper

In [0]:
ingest_util = SleeperIngestUtil(year = par_year, matchup_week = par_matchup_week, league_id = par_league_id)

## v1/league

In [0]:
if par_matchup_week > 0:

    data = ingest_util.read_api(f"https://api.sleeper.app/v1/league/{par_league_id}/")

    # add metadata
    data = ingest_util.add_metadata([data])
    # Write JSON data to a volume with a unique timestamp

    endpoint_name = 'league'
    ingest_util.write_data(data, endpoint_name)
    # dbutils.fs.put(f"{tgt_dir}/league/{par_year}/league_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

## /v1/league/*

In [0]:
if par_matchup_week > 0:
    endpoints = ['winners_bracket', 'losers_bracket']
    # endpoints = ['rosters', f'matchups/{par_matchup_week}', 'users', f'transactions/{par_matchup_week}', 'traded_picks', 'winners_bracket', 'losers_bracket']

    for endpoint in endpoints:

        # Fetch data from the API
        data = ingest_util.read_api(f"https://api.sleeper.app/v1/league/{par_league_id}/{endpoint}")

        # add metadata
        data = ingest_util.add_metadata(data)
        # Write JSON data to a volume with a unique timestamp
        endpoint_name = endpoint.split('/')[0]

        ingest_util.write_data(data, endpoint_name)
        # dbutils.fs.put(f"{tgt_dir}/{endpoint_dir}/{par_year}/{endpoint_dir}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

## v1/drafts

In [0]:
if par_matchup_week == 0:

    league_endpoints = ['drafts']

    for endpoint in league_endpoints:

        # Fetch data from the API
        data = ingest_util.read_api(f"https://api.sleeper.app/v1/league/{par_league_id}/{endpoint}")

        # add metadata
        data = ingest_util.add_metadata(data)
        # Write JSON data to a volume with a unique timestamp
        endpoint_name = endpoint.split('/')[0]

        ingest_util.write_data(data, endpoint_name)
        # dbutils.fs.put(f"{tgt_dir}/{endpoint_dir}/{par_year}/{endpoint_dir}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

        latest_draft_id = data[0]['draft_id']

        data = ingest_util.read_api(f"https://api.sleeper.app/v1/draft/{latest_draft_id}/picks")

        data = ingest_util.add_metadata(data)

        ingest_util.write_data(data, endpoint_name + '_picks')
        # dbutils.fs.put(f"{tgt_dir}/{endpoint_dir + '_picks'}/{par_year}/{endpoint_dir + '_picks'}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

        data = ingest_util.read_api(f"https://api.sleeper.app/v1/draft/{latest_draft_id}/traded_picks")

        data = ingest_util.add_metadata(data)

        ingest_util.write_data(data, endpoint_name + '_traded_picks')

        endpoints = ['users']

        for endpoint in endpoints:

            # Fetch data from the API
            data = ingest_util.read_api(f"https://api.sleeper.app/v1/league/{par_league_id}/{endpoint}")
            
            # add metadata
            data = ingest_util.add_metadata(data)
            # Write JSON data to a volume with a unique timestamp
            endpoint_name = endpoint.split('/')[0]

            ingest_util.write_data(data, endpoint_name)
            # dbutils.fs.put(f"{tgt_dir}/{endpoint_dir}/{par_year}/{endpoint_dir}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)
